In [1]:
import pandas as pd
import numpy as np
import requests
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

from collections import Counter

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
plt.style.use("seaborn")

/home/pnietoramos/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Defining Features and Classifier

In [2]:
df_all = pd.read_csv("Trees_clean.csv", index_col=False)

In [3]:
df_all.head(1)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type
0,2596,51,3,258,0,510,221,232,148,6279,5,29


In [4]:
X = df_all.drop("Cover_Type",axis=1)
y = df_all["Cover_Type"]

In [5]:
#Splitting the set (test = 0.3) in a stratified manner
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30,stratify=y, random_state=4444)

In [6]:
#scaling the features
sca = StandardScaler()
sca.fit_transform(X_train)
sca.transform(X_test);

In [7]:
RandomForestClassifier().get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

### Now without the distance to fire points

In [8]:
col_in = ["Elevation","Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points", "Soil_Type",
          "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"]
X_in = X[col_in]
X_train_in = X_train[col_in]
X_test_in = X_test[col_in]

In [9]:
rf = RandomForestClassifier(max_leaf_nodes=1000000, max_depth=50, n_estimators=200)
rf.fit(X_train_in, y_train)
y_pred = rf.predict(X_test_in)

In [10]:
print("Recall for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_test[y_test==(i+1)])))

Recall for cover:
Type 1:96.10083081570997
Type 2:97.487969314398
Type 3:95.66473988439306
Type 4:83.25242718446601
Type 5:82.51404494382022
Type 6:88.65642994241843
Type 7:95.38436535023565


In [11]:
print("Precision for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_pred[y_pred==(i+1)])))

Precision for cover:
Type 1:96.76315414230714
Type 2:96.38341185366137
Type 3:94.12034489084571
Type 4:86.61616161616162
Type 5:93.9624150339864
Type 6:92.91892979279822
Type 7:96.91215323645972


### Training with the whole set

In [12]:
rf = RandomForestClassifier(max_leaf_nodes=1000000, max_depth=50, n_estimators=200)
rf.fit(X_in, y)
y_pred = rf.predict(X_in)

In [13]:
print("Recall for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y, y_pred)[i,i]/len(y[y==(i+1)])))

Recall for cover:
Type 1:100.0
Type 2:100.0
Type 3:100.0
Type 4:100.0
Type 5:100.0
Type 6:100.0
Type 7:100.0


In [14]:
print("Precision for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y, y_pred)[i,i]/len(y_pred[y_pred==(i+1)])))

Precision for cover:
Type 1:100.0
Type 2:100.0
Type 3:100.0
Type 4:100.0
Type 5:100.0
Type 6:100.0
Type 7:100.0


### Predictions and visualizations

["Elevation","Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points",
"Soil_Type","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"]

In [69]:
sca_all = StandardScaler()
sca_all.fit_transform(X_in);

Predicting the coverage with an example predicted to be of high risk with our fire risk model.

In [70]:
example = np.array([2500, 1500, 0, 11, 200, 50])
sca_all.transform(example.reshape(1,-1))
rf.predict(example.reshape(1,-1))

/home/pnietoramos/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([2])

It will naturally grow Lodgepole Pine

In [71]:
example = np.array([2500, 750,0, 11, 800, 50])
sca_all.transform(example.reshape(1,-1))
rf.predict(example.reshape(1,-1))

/home/pnietoramos/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([3])

It will naturally grow Ponderosa Pine